# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: Ing. en Sistemas Computacionales** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 07**: Real-time log analyzer

**Fecha**: 03 de abril del 2025

**Nombre del Estudiante**: Luis, Benja, Sam

**Profesor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Storage-Solutions") \
    .master("spark://e0047bbfd1d0:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/04 04:22:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Creación del FileStream

In [3]:
logs = spark \
                .readStream \
                .format("text") \
                .option("maxFilesPerTrigger", 1) \
                .load("/home/jovyan/notebooks/data/log/input/")

In [4]:
from pyspark.sql.functions import split

logs_df = logs.select(split(logs.value, r'\s*\|\s*').alias("logs_array"))
logs_df = logs_df.withColumn("date", logs_df["logs_array"].getItem(0).cast("date"))
logs_df = logs_df.withColumn("type", logs_df["logs_array"].getItem(1).cast("string"))
logs_df = logs_df.withColumn("details", logs_df["logs_array"].getItem(2).cast("string"))
logs_df = logs_df.withColumn("server", logs_df["logs_array"].getItem(3).cast("string"))


logs_df.printSchema()

root
 |-- logs_array: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- date: date (nullable = true)
 |-- type: string (nullable = true)
 |-- details: string (nullable = true)
 |-- server: string (nullable = true)



In [5]:
logs_df = logs_df.filter(logs_df.type == 'Error')

In [6]:
from gatubelxs.query_listener import QueryListener

spark.streams.addListener(QueryListener())

In [ ]:
query = logs_df \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='2 seconds') \
                .format("console") \
                .option("truncate", "false") \
                .start()

query.awaitTermination(10)

25/04/04 04:23:26 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-93d80201-6a9f-4823-a4a3-d77ebe6d6c15. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/04 04:23:26 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


Query started: fc4b03dc-7e3d-48ab-bcdc-a4edbc1143dc


False

-------------------------------------------
Batch: 0
-------------------------------------------
+----------------------------------------------------------------------+----------+-----+-------------------------+-------------+
|logs_array                                                            |date      |type |details                  |server       |
+----------------------------------------------------------------------+----------+-----+-------------------------+-------------+
|[2025-04-04 03:46:35, Error, 500 Internal Server Error, server-node-1]|2025-04-04|Error|500 Internal Server Error|server-node-1|
|[2025-04-04 03:46:35, Error, 500 Internal Server Error, server-node-2]|2025-04-04|Error|500 Internal Server Error|server-node-2|
|[2025-04-04 03:46:35, Error, 500 Internal Server Error, server-node-1]|2025-04-04|Error|500 Internal Server Error|server-node-1|
|[2025-04-04 03:46:35, Error, 500 Internal Server Error, server-node-1]|2025-04-04|Error|500 Internal Server Error|server-n

25/04/04 04:23:48 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 2000 milliseconds, but spent 20520 milliseconds


...

-------------------------------------------
Batch: 4
-------------------------------------------
+----------------------------------------------------------------------+----------+-----+-------------------------+-------------+
|logs_array                                                            |date      |type |details                  |server       |
+----------------------------------------------------------------------+----------+-----+-------------------------+-------------+
|[2025-04-04 03:46:56, Error, 500 Internal Server Error, server-node-1]|2025-04-04|Error|500 Internal Server Error|server-node-1|
|[2025-04-04 03:46:56, Error, 500 Internal Server Error, server-node-1]|2025-04-04|Error|500 Internal Server Error|server-node-1|
|[2025-04-04 03:46:56, Error, 500 Internal Server Error, server-node-1]|2025-04-04|Error|500 Internal Server Error|server-node-1|
|[2025-04-04 03:46:56, Error, 500 Internal Server Error, server-node-2]|2025-04-04|Error|500 Internal Server Error|server-n

25/04/04 04:24:02 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 2000 milliseconds, but spent 5672 milliseconds


Data volume is high
-------------------------------------------
Batch: 5
-------------------------------------------
+----------------------------------------------------------------------+----------+-----+-------------------------+-------------+
|logs_array                                                            |date      |type |details                  |server       |
+----------------------------------------------------------------------+----------+-----+-------------------------+-------------+
|[2025-04-04 03:47:01, Error, 500 Internal Server Error, server-node-1]|2025-04-04|Error|500 Internal Server Error|server-node-1|
|[2025-04-04 03:47:01, Error, 500 Internal Server Error, server-node-2]|2025-04-04|Error|500 Internal Server Error|server-node-2|
|[2025-04-04 03:47:01, Error, 500 Internal Server Error, server-node-1]|2025-04-04|Error|500 Internal Server Error|server-node-1|
|[2025-04-04 03:47:01, Error, 500 Internal Server Error, server-node-1]|2025-04-04|Error|500 Internal S

25/04/04 04:24:04 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 2000 milliseconds, but spent 2384 milliseconds


Data volume is high


In [ ]:
sc.stop()